<a href="https://colab.research.google.com/github/mao-code/SLFN_WeightTuning/blob/main/SLFN_WeightTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This is based on the HW3 of New Learning Algorithm class.
<img 
width="500px"
src="https://lh3.googleusercontent.com/fife/APg5EOZwrOY0oLYi8YBH38hc_wuinI82mVPRcphGYdR72lxn9aZISGLKs1_OLCXGPF90SVd3hpzHRq9MCxgvCq6IGSyv2winW34Qj16nBvYEddYIvwzQ6hYaWH1kAgVXnfp8FE_e5beZTKDxXmK6dfLwrrYj-G1POJSfaU-Q7VCM1392hTfOqSsS8ZQQQhgV7cYh9mWDVwnhNcTE3qRoqjX_8FceEgiQYUurcNysmEsiGgnB7NkFzoyulDWTYXqBxjJU5T1lMvWCOIliO3lnfcUZIQ24dPj2RnLge95p4axVNBwTq7ZY8BsM0S0FrnBe_fzEY6Nm6Qbl24RXPQXUuP0S7SbIgz1BEkbvjzB5AY9r3o9GUS9Bk5idLqULNOZq9or9onL5Z8O2C_Tkwx1s660x3twZgeF4m3BUPBpjSjmckdCBZFCBeIM-OyG45FJJ7pcRz1V_OPUouQHLWbgi70vGY6BiOBpZcR39Agq_XRNAH5ZTHemKJN9Vz1lbd_FAAldOfdINCM0JJoh_RWvOYuJYO_XolnatONqvsQBxGRwLD-6J_fdLOCec5G8CA42xfs1ycM5mNcc24bVG8Q4dyuetbEF2jJ0qahu1LplQY4YM2rklkcWqAeXm0Qu119gd0lZBhucXDn5lamLXBA5h1xTeFXVsgKuW75hElDjIbBjGt2mL-qIMq1Kw2t0uilsVQhSLv6ILo8eYtOWpvXO2aqmfRPpGcVv1mr0mfILRTNr6VTuLMSVaLFGX0ptBGfLn96xo5KDZPQid29oWQZnWvW-cw4OMzH43i-kqcRQOxT0PlXKLbqcCDFj8Bxb2ccD4xpcQQFb6yb62bMHTDrYd5BcxT9HpHJ5ZtHDI8sEPIOFeP8RtvxURLLKWC2vLRk46U6EkgvMtGteTA_aK07mmY0i1JOLElewJF3lP-5E6r0crBaFXO8v8tTSI6Ps4GL3d3Dkil0Q9NMua0n3piM-J2vg1te6e3QBhD0EwAP_Gj7EQ8EgAARnjyLMOG9QoQJb06PYbex_nvvXye5kIvoG-B_538XUjIMVGiAiDyVDnydMDPtaB6ZD4f0EK3t5DZ81IMjpYvnuDycm9EQ0OsR6jlJe23eBAm63ECsDgJo5rUOYldTFOnE4vtaP4T30ELhGCz_ti8hXKAe7Bu1-zUFvc131lHC0SJUAitxLQLc0GjO1t2nRvNXV1Yk1qKzL68aHzGKGR8L6OpMX7qAQ2sp25POhw2l4s0MKG3nPn-e9FaLvg4TgOQj-sw-eqGCvTf834tzI0ldJzSIHWpmWKPAMDR5wKVVWDm2lMIgGbQE28-LajH1FR9-qKqsFqVSrmmtvUYBQVMhwLS2_yQwtjBQzv51Rxa0PCJViFER9ZCgNaBG5EUcMrepiANCuBAXy_Fc64pbHGOD4_7aaYSgT-n53ql_3mqo0FtTDzAyeIdAvP08ib_0pH9jraUiFTw8CzAk_yNRWbM4zwDLzpHASFpFZRT6mISWCUVPQcrqSSMGDirtoUXej0qtB0yZESGJbsXweATfx1GrECWLRH25aHLxasYmRK_-ShPfl6INC6yYqEr5dG5AvFY0MH2ihXmZNxFnZPdFuWoVA=w2992-h1654" alt="HW3"></img>

In [ ]:
import random
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader

## Setup Seed
在深度學習中，許多操作都涉及到隨機性，例如權重初始化、dropout、數據增強等。這些操作在沒有指定隨機種子的情況下，每次運行時都會生成不同的隨機數序列，從而導致不同的結果。而通過設定隨機種子，可以使得每次生成的隨機數序列都是相同的，從而使得程式的運行結果穩定不變。

在一般學習、測試模型的時候，我們並不需要關心如何固定模型的參數，好讓模型可以重現 —— 但今天若是在科學實驗的背景下，如何讓一組模型的實驗可以重現，就是一件非常重要的事情了。

In [ ]:
# In order to reproduce the result of the model.
def setup_seed(seed):
   torch.manual_seed(seed)
   torch.cuda.manual_seed(seed)
   torch.cuda.manual_seed_all(seed)
   np.random.seed(seed)
   random.seed(seed)
   torch.backends.cudnn.benchmark = False
   torch.backends.cudnn.deterministic = True
   torch.backends.cudnn.enabled = True

setup_seed(8888)

## Data Preprocessing

### Load Data

In [ ]:
# use colab default dataset (parse into pandas dataframe)
housing_train_df = pd.read_csv('sample_data/california_housing_train.csv')
housing_test_df = pd.read_csv('sample_data/california_housing_test.csv')

### Normalize and Show Data

In [ ]:
housing_df = pd.concat([housing_train_df, housing_test_df], ignore_index = True)

# Normalizing. In order to get prettier result
housing_df = (housing_df - housing_df.mean()) / housing_df.std() 

housing_df

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,2.623352,-0.672591,-1.083095,1.366966,1.771168,-0.362989,-0.072107,-1.251624,-1.213104
1,2.543496,-0.574283,-0.765186,2.303412,3.240388,-0.262197,-0.095684,-1.079867,-1.098875
2,2.498578,-0.906658,-0.924141,-0.880871,-0.865347,-0.965969,-1.002085,-1.168851,-1.050414
3,2.493587,-0.930065,-1.162572,-0.522007,-0.477834,-0.805056,-0.716543,-0.358055,-1.156855
4,2.493587,-0.962834,-0.685709,-0.543603,-0.503985,-0.708685,-0.622235,-1.024614,-1.225219
...,...,...,...,...,...,...,...,...,...
19995,-0.146649,-0.564920,-0.447278,-0.545441,0.247267,-0.148144,0.281546,-1.417172,0.155051
19996,0.711802,-0.733449,-0.129369,1.203846,1.293314,1.830551,1.405379,-0.253390,0.260627
19997,-0.066793,0.315171,-1.480481,-0.772430,-0.801158,-0.647680,-0.732261,-0.832808,-1.255507
19998,1.220883,-0.714723,0.903834,-1.167594,-1.245728,-1.219716,-1.271910,-0.316431,-0.385807


In [ ]:
print(housing_df.loc[housing_df["median_house_value"].idxmax(), "median_house_value"]) # max median_house_value
print(housing_df.loc[housing_df["median_house_value"].idxmin(), "median_house_value"]) # min median_house_value

2.53483684816792
-1.6622413519178973


### Split Training(80%) and Testing(20%) Dataset

In [ ]:
from sklearn.model_selection import train_test_split 

# random sample
# to reproduce result, give random_state
# housing_train_df, housing_test_df = train_test_split(housing_df, test_size=0.2, random_state=42)

# order sample
housing_train_df = housing_df.loc[:15999]
housing_test_df = housing_df.loc[16000:]

print(f"Total {len(housing_df)}, Training: {len(housing_train_df)}, Testing: {len(housing_test_df)}")

Total 20000, Training: 16000, Testing: 4000


## Initialization

In [ ]:
class HousingDataset(Dataset): # inherit Dataset

  def __init__(self, dataframe):
    self.dataframe = dataframe

  def __getitem__(self, index):
    row = self.dataframe.iloc[index].to_numpy().astype('float32') # using index to locate a data row
    features = row[0:-1] # get all columns except last one
    label = row[-1] # last column => we want to predict
    return features, label

  def __len__(self):
    return len(self.dataframe)

In [ ]:
setup_seed(8888)

train_dataset = HousingDataset(housing_train_df.astype('float32'))
test_dataset = HousingDataset(housing_test_df.astype('float32'))
train_dataloader = DataLoader(train_dataset, batch_size = len(housing_train_df), shuffle = True)
test_dataloader = DataLoader(test_dataset, batch_size = len(housing_test_df), shuffle = True)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

Using cuda device


## Two Layers NN model
<img
width="400px"
src="https://lh3.google.com/u/1/d/1XtrVTTKHxHkQNOpvzep27KpEymNAOY7I=w2992-h1654-iv2"></img>

In [ ]:
class TwoLayerNNModel(torch.nn.Module):

  def __init__(self, D_in, H, D_out, activation_func, initial_weights = None):
    super(TwoLayerNNModel, self).__init__()
    self.linear1 = torch.nn.Linear(D_in, H) # FC1 (fully connected)
    self.BatchNorm1 = torch.nn.BatchNorm1d(H) # BN1 (batch normalization)
    self.linear2 = torch.nn.Linear(H, D_out)
    
    self.activation_func = activation_func
    if activation_func == 'ReLU':
      self.relu = torch.nn.ReLU()
    if activation_func == 'Tanh':
      self.tanh = torch.nn.Tanh()

    self.initial_weights = initial_weights
    if self.initial_weights == 'small random number':
      # H = W * X + B ; X = D_in
      self.linear1.weight = torch.nn.Parameter(0.01 * torch.FloatTensor(np.random.randn(H, D_in))) # transpose
      self.linear2.weight = torch.nn.Parameter(0.01 * torch.FloatTensor(np.random.randn(D_out, H)))
    
    #the other two ways to initialize weight
    if self.initial_weights == 'Xavier':
      torch.nn.init.xavier_uniform_(self.linear1.weight)
      torch.nn.init.xavier_uniform_(self.linear2.weight)
    if self.initial_weights == 'Kaiming':
      torch.nn.init.kaiming_uniform_(self.linear1.weight)
      torch.nn.init.kaiming_uniform_(self.linear2.weight)
  
  # input -> FC1 -> BN1 -> ReLU -> FC2 -> output
  def forward(self, x): # input
    layer1 = self.linear1(x) # FC1
    
    batchnorm1 = self.BatchNorm1(layer1) # BN1

    # activation
    activation = None
    if self.activation_func == 'ReLU':
      activation = self.relu(batchnorm1)
    if self.activation_func == 'Tanh':
      activation = self.tanh(batchnorm1)
    
    pred = self.linear2(activation) # FC2
    return pred

## Initial Model Settings

In [ ]:
D_in = len(housing_train_df.columns) - 1 # all features except the last one(label)
D_out = 1 # predict the label(median_house_value)
hidden_nodes = [5, 8, 11] # different numbers of hidden nodes. can choose what you want
learning_rate = 1e-3 # init learning rate
loss_fn = torch.nn.MSELoss().to(device) # loss function using Mean Square Error(MSE)

## Weight_Tuning_LG_UA
Learning Goal and Undesired Atractor

<img
width="800px"
src="https://lh3.google.com/u/1/d/1ixKGpLJgH1N9BNlaYO-OQadTsrVSzdA_=w2070-h1654-iv2"></img>

1. c is for c_th input data
2. epsilon is 0.9905 (tolerant loss) => LG
3. initial learning rate is 0.001
4. epsilon1 is 0.00001 (tolerant min of learning rate) => UA
  * may be caused by local min, saddle point, etc.

Note: As soon as you meet the loop(at first entery point), build and loop.

In [ ]:
setup_seed(8888)

model_LG_UA = TwoLayerNNModel(D_in, hidden_nodes[1], D_out, 'Tanh', 'small random number').to(device)
optimizer_LG_UA = torch.optim.Adam(model_LG_UA.parameters(), lr = learning_rate) # Use Adam optimizer

# Adam => Adaptive Moment Estimation => optimized gradient based optimizer

### Training

In [ ]:
# we set one batch = dataset size, so it gonna loop onece
for batch, (X, y) in enumerate(train_dataloader):

  t = 0
  isLGSatisfied = False
  isMeetUA = False
  
  epsilon = 0.9905
  epsilon1 = 1e-5

  # forward
  print(f"{t}th round: ")

  print('initial forward operation: ')
  pred = model_LG_UA(X.to(device))
  loss = loss_fn(pred, y.to(device))
  print('* total loss: ', loss.item())
  print('-------------------------------------')
  
  # Store w first
  original_param = {}
  for param in model_LG_UA.parameters():
    original_param[param] = param

  # weight tuning
  # loop1
  while not isLGSatisfied:
    t+=1
    print(f"{t}th round (now at LG): ")
    print('check stopping criteria(LG): ')
    
    if loss.item() < epsilon:
      print("LG OK!")
      isLGSatisfied = True
      break

    else:
      print("LG not OK!")
     
      # if put ouside the loop, need to retain graph
      print('Backward: calculate direction')
      loss.backward() # calculate direction

      # loop2
      while not isMeetUA:
        print('weight adjustment')
        optimizer_LG_UA.step()

        print("Forward Operation")
        pred2 = model_LG_UA(X.to(device))
        loss2 = loss_fn(pred2, y.to(device))

        print("check if loss is less than the original one")
        
        if loss2.item() < loss.item():
          print("\nloss becomes smaller!")

          print('lr = lr * 1.2')
          optimizer_LG_UA.param_groups[0]['lr'] *= 1.2

          print(f"update loss: {loss2}")
          loss = loss2

          print('-------------------------------------')

          # clean gradient
          optimizer_LG_UA.zero_grad()

          break

        else:
          print("\nloss becomes bigger!")

          t+=1
          print(f"\n{t}th round (now at UA)")
          print("check stopping criteria(UA): ")
          if optimizer_LG_UA.param_groups[0]['lr'] > epsilon1:
            print("haven't met UA!")

            # restore w
            print("restore w")
            for param in model_LG_UA.parameters():
              param = original_param[param]

            # step smaller
            print('lr = lr * 0.7')
            optimizer_LG_UA.param_groups[0]['lr'] *= 0.7 

            print('-------------------------------------')

            # clean gradient
            optimizer_LG_UA.zero_grad()

            continue
            
          else:
            print("have met UA!")
            print('-------------------------------------')
            isMeetUA = True
          
    if isMeetUA:
      break

  if isLGSatisfied == True:
    print('\n-------------------------------------')
    print("Acceptable SLFN")
  else:
    print('\n-------------------------------------')
    print("Unacceptable SLFN")

  print(f"loss: {loss}")

0th round: 
initial forward operation: 
* total loss:  0.9949537515640259
-------------------------------------
1th round (now at LG): 
check stopping criteria(LG): 
LG not OK!
Backward: calculate direction
weight adjustment
Forward Operation
check if loss is less than the original one

loss becomes smaller!
lr = lr * 1.2
update loss: 0.9947198629379272
-------------------------------------
2th round (now at LG): 
check stopping criteria(LG): 
LG not OK!
Backward: calculate direction
weight adjustment
Forward Operation
check if loss is less than the original one

loss becomes smaller!
lr = lr * 1.2
update loss: 0.9944754838943481
-------------------------------------
3th round (now at LG): 
check stopping criteria(LG): 
LG not OK!
Backward: calculate direction
weight adjustment
Forward Operation
check if loss is less than the original one

loss becomes smaller!
lr = lr * 1.2
update loss: 0.994215190410614
-------------------------------------
4th round (now at LG): 
check stopping crit

/usr/local/lib/python3.9/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([16000])) that is different to the input size (torch.Size([16000, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)



loss becomes smaller!
lr = lr * 1.2
update loss: 0.9930391311645508
-------------------------------------
8th round (now at LG): 
check stopping criteria(LG): 
LG not OK!
Backward: calculate direction
weight adjustment
Forward Operation
check if loss is less than the original one

loss becomes smaller!
lr = lr * 1.2
update loss: 0.9926501512527466
-------------------------------------
9th round (now at LG): 
check stopping criteria(LG): 
LG not OK!
Backward: calculate direction
weight adjustment
Forward Operation
check if loss is less than the original one

loss becomes smaller!
lr = lr * 1.2
update loss: 0.9922142624855042
-------------------------------------
10th round (now at LG): 
check stopping criteria(LG): 
LG not OK!
Backward: calculate direction
weight adjustment
Forward Operation
check if loss is less than the original one

loss becomes smaller!
lr = lr * 1.2
update loss: 0.9917640089988708
-------------------------------------
11th round (now at LG): 
check stopping criter

## Weight_Tuning_EB_LG_UA
Epoch Bound, Learning Goal and Undesired Atractor

<img
width="800px"
src="https://lh3.google.com/u/1/d/1qjI9MxwToHkw0k8t_pXCavMz8AGq10sz=w2070-h1654-iv2"></img>

In [ ]:
setup_seed(8888)
epoch_bound = 50

model_EB_LG_UA = TwoLayerNNModel(D_in, hidden_nodes[1], D_out, 'Tanh', 'small random number').to(device)
optimizer_EB_LG_UA = torch.optim.Adam(model_LG_UA.parameters(), lr = learning_rate) 

In [ ]:
# we set one batch = dataset size
# so it gonna loop onece (every data => one epoch)
for batch, (X, y) in enumerate(train_dataloader):

  t = 0 
  isLGSatisfied = False
  isMeetUA = False
  isMeetEpochBound = False
  
  epsilon = 0.9905
  epsilon1 = 1e-5

  # forward
  print(f"{t}th round: ")

  print('initial forward operation: ')
  pred = model_LG_UA(X.to(device))
  loss = loss_fn(pred, y.to(device))
  print('* total loss: ', loss.item())
  print('-------------------------------------')
  
  # Store w first
  original_param = {}
  for param in model_LG_UA.parameters():
    original_param[param] = param

  # weight tuning
  # loop1
  while not isLGSatisfied:
    print(f"{t+1}th round (now at LG)") # Cuz begining t = 0
    print('check stopping criteria(LG): ')
    
    if loss.item() < epsilon:
      print("LG OK!")
      isLGSatisfied = True
      break

    else:
      print("LG not OK!")
     
      # if put ouside the loop, need to retain graph
      print('Backward: calculate direction')
      loss.backward() # calculate direction

      # loop2
      while not isMeetUA:
        print('weight adjustment')
        optimizer_LG_UA.step()

        print("Forward Operation")
        pred2 = model_LG_UA(X.to(device))
        loss2 = loss_fn(pred2, y.to(device))

        print("check if loss is less than the original one")
        
        if loss2.item() < loss.item():
          print("\nloss becomes smaller!")

          print('lr = lr * 1.2')
          optimizer_LG_UA.param_groups[0]['lr'] *= 1.2

          print(f"update loss: {loss2}")
          loss = loss2

          # successfully weight tune once
          t+=1
          isMeetEpochBound = t == epoch_bound

          if isMeetEpochBound:
            print("Meet Epoch Bound!")
            break

          print('-------------------------------------')

          # clean gradient
          optimizer_LG_UA.zero_grad()

          break

        else:
          print("\nloss becomes bigger!")

          print(f"\n{t}th round (now at UA)")
          print("check stopping criteria(UA): ")
          if optimizer_LG_UA.param_groups[0]['lr'] > epsilon1:
            print("haven't met UA!")

            # restore w
            print("restore w")
            for param in model_LG_UA.parameters():
              param = original_param[param]

            # step smaller
            print('lr = lr * 0.7')
            optimizer_LG_UA.param_groups[0]['lr'] *= 0.7 

            print('-------------------------------------')

            # clean gradient
            optimizer_LG_UA.zero_grad()

            continue
            
          else:
            print("have met UA!")
            print('-------------------------------------')
            isMeetUA = True
          
    if isMeetUA:
      break

    if isMeetEpochBound:
      break

  if isLGSatisfied == True:
    print('\n-------------------------------------')
    print("Acceptable SLFN")
  else:
    print('\n-------------------------------------')
    print("Unacceptable SLFN")

  print(f"loss: {loss}")

0th round: 
initial forward operation: 
* total loss:  0.9904861450195312
-------------------------------------
1th round (now at LG)
check stopping criteria(LG): 
LG OK!

-------------------------------------
Acceptable SLFN
loss: 0.9904861450195312


## Export Model
* .pt 
  * save the state of pytorch model (including parameters)
  * can load later and train

* .pth
  * store pre-trained model weights that can be loaded into a PyTorch model
  * fine-tune the model on your own dataset

In [ ]:
torch.save(model_LG_UA, 'model_LG_UA.pt')
torch.save(model_EB_LG_UA, 'model_EB_LG_UA.pt')